Text summarization using NLP

1.Data Loading and Initial Exploration:

In [6]:
import pandas as pd
import numpy as np
import nltk
import re
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [8]:
from google.colab import files
uploaded = files.upload()

Saving tennis_articles.csv to tennis_articles (1).csv


In [9]:
df = pd.read_csv('tennis_articles (1).csv', encoding = "ISO-8859-1")

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   article_id     8 non-null      int64 
 1   article_title  8 non-null      object
 2   article_text   8 non-null      object
 3   source         8 non-null      object
dtypes: int64(1), object(3)
memory usage: 388.0+ bytes


In [11]:
df.head(10)

,article_id,article_title,article_text,source
0,1,"I do not have friends in tennis, says Maria Sh...",Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,Federer defeats Medvedev to advance to 14th Sw...,"BASEL, Switzerland (AP)  Roger Federer advanc...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Tennis: Roger Federer ignored deadline set by ...,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Nishikori to face off against Anderson in Vien...,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,Roger Federer has made this huge change to ten...,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...
5,6,Rafael Nadal: World No 1 ARRIVES for Paris Mas...,Nadal has not played tennis since he was force...,https://www.express.co.uk/sport/tennis/1037119...
6,7,"TENNIS.COM PODCAST: POINT DEFENSE, RANKING DRO...","Tennis giveth, and tennis taketh away. The end...",http://www.tennis.com/pro-game/2018/10/tennisc...
7,8,Tennis journalists heartbreaking insight on T...,I PLAYED golf last week with Todd Reid. He pic...,https://www.foxsports.com.au/tennis/tennis-jou...


In [12]:
df.columns

Index(['article_id', 'article_title', 'article_text', 'source'], dtype='object')

In [13]:
df.drop('article_title', axis='columns')

,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP)  Roger Federer advanc...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...
5,6,Nadal has not played tennis since he was force...,https://www.express.co.uk/sport/tennis/1037119...
6,7,"Tennis giveth, and tennis taketh away. The end...",http://www.tennis.com/pro-game/2018/10/tennisc...
7,8,I PLAYED golf last week with Todd Reid. He pic...,https://www.foxsports.com.au/tennis/tennis-jou...


2.Text Preprocessing:

--> tokenisation

In [14]:
from nltk.tokenize import sent_tokenize

In [15]:
df["sentences"] = df["article_text"].apply(sent_tokenize)

In [16]:
# Tokeniser chaque article en phrases et les stocker dans une seule liste
sentences = [sentence for article in df["article_text"] for sentence in sent_tokenize(article)]

In [17]:
len(sentences)

130

In [18]:
print(sentences[:5])

['Maria Sharapova has basically no friends as tennis players on the WTA Tour.', "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.", 'I think everyone knows this is my job here.', "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.", "So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match."]


-->Glove word embeddings

In [19]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2025-03-14 20:11:34--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-03-14 20:11:35--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-03-14 20:11:35--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [20]:
# Initialiser un dictionnaire pour stocker les embeddings
word_embeddings = {}

# Définir le chemin du fichier (nous utilisons la version 100 dimensions)
glove_path = "glove.6B.100d.txt"

# Charger les embeddings dans un dictionnaire
with open(glove_path, "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]  # Le premier élément est le mot
        coefs = np.asarray(values[1:], dtype='float32')  # Le reste est le vecteur du mot
        word_embeddings[word] = coefs  # Stocker dans le dictionnaire

# Vérification
print("\n Embeddings GloVe chargés avec succès !")
print("Exemple d'embedding pour 'tennis' :", word_embeddings["tennis"][:10])  # Afficher les 10 premières valeurs
# Les embeddings sont maintenant prêts à être utilisés pour représenter nos phrases sous forme de vecteurs !


 Embeddings GloVe chargés avec succès !
Exemple d'embedding pour 'tennis' : [ 0.21508   0.61981   0.84039   0.71394  -0.29904   0.56481   0.18241
  0.76767  -0.75897  -0.056711]


-->Clean the sentences by removing punctuation, special characters, and numbers.

In [21]:
from nltk.corpus import stopwords
nltk.download('stopwords')  # Télécharger la liste des stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
# Charger la liste des stopwords en anglais
stop_words = set(stopwords.words('english'))

In [23]:
# créer une fonction qui applique ces transformations:
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Supprimer tout sauf les lettres et espaces
    text = text.lower()  # Convertir en minuscules
    words = text.split()  # Découper la phrase en mots
    words = [word for word in words if word not in stop_words]  # Supprimer les stopwords
    return " ".join(words)


In [24]:
#  Nettoyer toutes les phrases avec une list comprehension
clean_sentences = [clean_text(sentence) for sentence in sentences]

In [25]:
print(clean_sentences[:5])  # Afficher les 5 premières phrases nettoyées

['maria sharapova basically friends tennis players wta tour', 'russian player problems openly speaking recent interview said dont really hide feelings much', 'think everyone knows job', 'im courts im court playing im competitor want beat every single person whether theyre locker room across net', 'im one strike conversation weather know next minutes go try win tennis match']


--> Vectorize sentences using the average of the word embeddings.

 Convertit une phrase en vecteur en faisant la moyenne des embeddings de ses mots.
 Arguments: sentence (str): Phrase nettoyée
        word_embeddings (dict): Dictionnaire contenant les embeddings GloVe
        embedding_dim (int): Taille des vecteurs d'embeddings (100 par défaut)
Returns: np.array: Vecteur moyen de la phrase.
   

In [26]:
def sentence_to_vector(sentence, word_embeddings, embedding_dim=100):
    words = sentence.split()  # Découper la phrase en mots
    word_vectors = [word_embeddings[word] for word in words if word in word_embeddings]  # Récupérer les embeddings connus

    if len(word_vectors) == 0:
        return np.zeros(embedding_dim)  # Retourner un vecteur nul si aucun mot n'est reconnu

    return np.mean(word_vectors, axis=0)  # Moyenne des vecteurs des mots d’une phrase pour obtenir un vecteur unique par phrase.)

#  Test sur une phrase simple
test_sentence = "Tennis is a great sport"
vector = sentence_to_vector(test_sentence, word_embeddings)
print("\nExemple de vecteur pour la phrase 'Tennis is a great sport':")
print(vector[:10])  # Afficher les 10 premières valeurs du vecteur
# Ce code convertit une phrase en un vecteur en utilisant la moyenne des embeddings GloVe des mots présents.


Exemple de vecteur pour la phrase 'Tennis is a great sport':
[-0.17898399  0.3147715   0.44038    -0.11655825  0.07391751  0.21331
  0.077589    0.278675   -0.257694   -0.11883427]


In [27]:
#  Vectoriser toutes les phrases avec une list comprehension
sentence_vectors = [sentence_to_vector(sentence, word_embeddings) for sentence in clean_sentences]

In [28]:
print(sentence_vectors[:2])

[array([ 5.1488996e-02,  1.1055850e-01,  6.9508630e-01,  1.8919174e-01,
       -9.5819749e-02,  3.2032904e-01,  2.7069587e-01,  5.4250866e-01,
       -3.0597627e-01, -1.5638362e-01,  3.7017390e-01,  8.0959372e-02,
        8.4149912e-03,  2.4760251e-01, -3.6938900e-01, -7.6114014e-02,
        8.0868371e-02,  2.3067249e-01, -2.7043623e-01,  5.1389265e-01,
       -6.1262503e-02,  3.8794887e-01,  1.0313426e-01,  7.7259129e-01,
        2.5999275e-01, -7.9616874e-02,  1.4216137e-01, -9.6276498e-01,
        7.5499874e-01,  6.0333610e-02, -4.5862746e-01,  2.3680991e-01,
        2.2918129e-01, -1.5647289e-01,  3.9768240e-01, -2.3275003e-02,
       -5.0558424e-01,  4.1330451e-01, -2.8579500e-01, -1.3524838e-01,
       -1.3711525e-01, -1.4899123e-01,  3.3757976e-01, -3.4958416e-01,
        1.5350363e-01, -2.3337083e-01, -1.9848561e-01, -1.2783751e-01,
        5.0812745e-01, -3.6868298e-01, -2.2850087e-01, -3.1534615e-01,
        1.3616650e-01,  2.2253501e-01,  1.1951500e-01, -1.7102875e+00,
     

3.Similarity Matrix and Graph Construction:

--> Calculate the cosine similarity between all pairs of sentence vectors.

La similarité cosinus mesure à quel point deux vecteurs sont alignés.
1 = phrases très similaires
0 = phrases sans lien
-1 = phrases opposées (rare en NLP)

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
#  1. Calculer la matrice de similarité cosinus
similarity_matrix = cosine_similarity(sentence_vectors)

In [33]:
print(similarity_matrix[:5, :5])  # Afficher les 5 premières lignes et colonnes
#  matrice où chaque valeur indique la similarité entre deux phrases

[[1.         0.6426971  0.59156992 0.72626793 0.7727943 ]
 [0.6426971  1.         0.85573618 0.81341243 0.83014321]
 [0.59156992 0.85573618 1.         0.78913021 0.79725081]
 [0.72626793 0.81341243 0.78913021 1.         0.88782728]
 [0.7727943  0.83014321 0.79725081 0.88782728 1.        ]]


In [34]:
import networkx as nx

In [35]:
# 1. Construire le graphe basé sur la matrice de similarité
graph = nx.from_numpy_array(similarity_matrix)

In [36]:
#  2. Appliquer l'algorithme PageRank pour classer les phrases
scores = nx.pagerank(graph)

In [37]:
# 3. Vérification
print("\n✅ Exemple de scores PageRank pour quelques phrases :")
sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
for i in range(5):
    print(f"Phrase {sorted_scores[i][0]} - Score: {sorted_scores[i][1]:.4f}")


✅ Exemple de scores PageRank pour quelques phrases :
Phrase 119 - Score: 0.0086
Phrase 39 - Score: 0.0085
Phrase 4 - Score: 0.0085
Phrase 66 - Score: 0.0085
Phrase 69 - Score: 0.0085


Génération du Résumé

1.Trier les phrases en fonction de leur score PageRank.
2.Sélectionner les N phrases les plus importantes pour créer le résumé.
3.Afficher le résumé final.

In [38]:
# 1. Trier les phrases en fonction du score PageRank (ordre décroissant)
sorted_sentences = sorted(scores.items(), key=lambda x: x[1], reverse=True)

In [39]:
# 2. Définir le nombre de phrases du résumé (ex: 5)
N = 5 # N phrases les plus importantes
summary_sentences = [clean_sentences[i[0]] for i in sorted_sentences[:N]] #clean_sentences[i[0]] permet d’obtenir le texte original des phrases les plus importantes.

In [43]:
# Assembler les phrases pour créer le résumé
summary = " ".join(summary_sentences)

In [44]:
print("\n **Résumé Généré :**")
print(summary)


 **Résumé Généré :**
nice trajectorythen reid recalledif hadnt got sick think could started pushing towards second week slams knows duringa comeback attempt five years later reid added bernard tomic us open federer slayer john millman list career scalps major players feel big event late november combined one january australian open mean much tennis little rest im one strike conversation weather know next minutes go try win tennis match felt like best weeks get know players playing fed cup weeks olympic weeks necessarily tournaments felt like really kind changed people little bit definitely lot quiet started become better meanwhile federer hoping improve service game hunts ninth swiss indoors title week


In [45]:
summary_sentences = [clean_sentences[i[0]] for i in sorted_sentences[:N]]
summary = ". ".join(summary_sentences) + "."
print("\n📢 **Résumé Final Amélioré :**")
print(summary)


📢 **Résumé Final Amélioré :**
nice trajectorythen reid recalledif hadnt got sick think could started pushing towards second week slams knows duringa comeback attempt five years later reid added bernard tomic us open federer slayer john millman list career scalps. major players feel big event late november combined one january australian open mean much tennis little rest. im one strike conversation weather know next minutes go try win tennis match. felt like best weeks get know players playing fed cup weeks olympic weeks necessarily tournaments. felt like really kind changed people little bit definitely lot quiet started become better meanwhile federer hoping improve service game hunts ninth swiss indoors title week.


In [48]:
#  Vérifier les phrases sélectionnées avant de les concaténer
print("\n **Phrases sélectionnées pour le résumé :**")
for i, sentence in enumerate(summary_sentences):
    print(f"{i+1}. {sentence}")


 **Phrases sélectionnées pour le résumé :**
1. nice trajectorythen reid recalledif hadnt got sick think could started pushing towards second week slams knows duringa comeback attempt five years later reid added bernard tomic us open federer slayer john millman list career scalps
2. major players feel big event late november combined one january australian open mean much tennis little rest
3. im one strike conversation weather know next minutes go try win tennis match
4. felt like best weeks get know players playing fed cup weeks olympic weeks necessarily tournaments
5. felt like really kind changed people little bit definitely lot quiet started become better meanwhile federer hoping improve service game hunts ninth swiss indoors title week
